In [1]:
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

In [2]:
from _momentum import Momentum

In [3]:
import random

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [4]:
data=pd.read_csv("MASI_2.csv",parse_dates=True, index_col=0)

In [5]:
data

,Open,High,Low,Close,Volume,Currency
Date,,,,,,
2002-01-03,3684.51,3694.96,3640.00,3640.00,0,MAD
2002-01-04,3641.08,3671.50,3640.00,3671.50,0,MAD
2002-01-07,3670.56,3674.71,3662.56,3667.30,0,MAD
2002-01-08,3655.56,3657.54,3639.32,3641.65,0,MAD
2002-01-09,3637.89,3645.03,3627.77,3644.19,0,MAD
...,...,...,...,...,...,...
2022-05-16,12682.21,12738.15,12648.59,12682.77,0,MAD
2022-05-17,12682.77,12733.07,12681.66,12716.65,0,MAD
2022-05-18,12716.65,12749.42,12672.62,12679.96,0,MAD


In [13]:
df=data["Close"]
high=data["High"]
low=data["Low"]

In [6]:
mom=Momentum(data)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## Fonction

In [12]:
premier=0 

In [13]:
def GeneticAlgo(dim,obj_func,MAX_GENERATIONS,BOUND_LOW,BOUND_UP,POPULATION_SIZE=40,P_CROSSOVER = 0.9,P_MUTATION = 0.1):
	global premier
	ONE_MAX_LENGTH=dim
	HALL_OF_FAME_SIZE = 5
	CROWDING_FACTOR=20
	RANDOM_SEED = 40
	random.seed(RANDOM_SEED)
	if premier> 0 :
		del creator.FitnessMax
		del creator.Individual
	premier=premier+1
	toolbox = base.Toolbox()
	toolbox.register("Num", random.randint, BOUND_LOW, BOUND_UP)
	creator.create("FitnessMax", base.Fitness, weights=(1.0,))
	creator.create("Individual", list, fitness=creator.FitnessMax)
	toolbox.register("individualCreator", tools.initRepeat, creator.Individual, toolbox.Num, ONE_MAX_LENGTH)
	toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)
	toolbox.register("evaluate", obj_func)
	toolbox.register("select", tools.selTournament, tournsize=3)
	toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=BOUND_LOW, up=BOUND_UP, eta=CROWDING_FACTOR)
	toolbox.register("mutate", tools.mutPolynomialBounded, low=BOUND_LOW, up=BOUND_UP, eta=CROWDING_FACTOR, indpb=0.20)
	def main():
		population = toolbox.populationCreator(n=POPULATION_SIZE)
		stats = tools.Statistics(lambda ind: ind.fitness.values)
		stats.register("max", np.max)
		stats.register("avg", np.mean)
		hof = tools.HallOfFame(HALL_OF_FAME_SIZE)
		population, logbook = algorithms.eaSimple(population, toolbox, cxpb=P_CROSSOVER, mutpb=P_MUTATION,ngen=MAX_GENERATIONS
			, stats=stats, halloffame=hof, verbose=True)
		print("Hall of Fame Individuals = ", *hof.items, sep="\n")
		print("Best Ever Individual = ", [ int(i) for i in hof.items[0]])
		maxFitnessValues, meanFitnessValues = logbook.select("max", "avg")
		sns.set_style("whitegrid")
		plt.plot(maxFitnessValues, color='red')
		plt.plot(meanFitnessValues, color='green')
		plt.xlabel('Génération')
		plt.ylabel('Max / Moyenne Fitness')
		plt.title('Max et Moyenne de  Fitness opar générations')
		plt.show()
	main()

In [14]:
# Coût des transactions
ti=0.0022
fr=0.0011
cb=0.0011
taux=ti+fr+cb

In [20]:
def obj_momentum(length):
    length=int(length[0])
    Mom=df.ta.mom(length)
    MOM=pd.Series(df.diff(length),name="MOM")
    MOMsignal=pd.Series(MOM.rolling(9, min_periods=9).mean(), name= "MOMsignal")
    #MOM=at.momentum(df,w,wsig=9)[["MOM","MOMsignal"]]
    signal=pd.DataFrame(index=df.index)
    signal["compa"]=np.nan
    signal["compa"][w:]=np.where(MOM["MOM"][w:] > MOM["MOMsignal"][w:],1,0)
    signal["signal"]=at.adjustsignal(signal["compa"].diff())
    cours=df*(1+taux*signal["signal"])
    pmval=at.pmv(cours,signal["signal"])
    return pmval.max(),

In [21]:
GeneticAlgo(1,obj_momentum,5,8,30)

AttributeError: 'Series' object has no attribute 'ta'

In [5]:
def momentum(df,w,wsig=9):
	"""
	Momentum
	  Paramètre: df: le  vecteur des prix
	              w: ordre
	              wsig ; ordre de signal ligne
	  Retour:   Momentum (pndas.DataFrame)
	"""
	MOM=pd.Series(df.diff(w),name="MOM")
	MOMsignal=pd.Series(MOM.rolling(wsig, min_periods=wsig).mean(), name= "MOMsignal")
	df=pd.DataFrame(df)
	df=df.join(MOM)
	df=df.join(MOMsignal)
	return df
